In [3]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163494 sha256=73a5f2c8d07669519067b6126e33cf7e2edf5f324fcb3de508f9d2730aeeef5b
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [5]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, accuracy

from google.colab import drive
drive.mount('/content/drive')

import warnings; warnings.simplefilter('ignore')

Mounted at /content/drive


# 단순 추천

In [10]:
md = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/Movie2/movies_metadata.csv")
md.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [11]:
print('////////////////////// Before ////////////////////// \n', md['genres'].head(), '\n////////////////////// Before //////////////////////')

# 딕셔너리안에 담겨있는 장르 정보를 리스트 형태로 세팅
# 1. md['genres'].fillna('[]') : genres 컬럼에 null 값을 '[](빈 리스트 값)'으로 채워넣음
# 2. apply(literal_eval) : literal_eval를 사용하여 String으로 되어있는 값을 List & Dictionary로 사용할 수 있게 변환
# 3. apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []) : x가 list인 경우 안에 들어있는 dictionary 중 name에 해당하는 값을 list에 담음
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

print('////////////////////// After ////////////////////// \n', md['genres'].head(), '\n////////////////////// After //////////////////////')

////////////////////// Before ////////////////////// 
 0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object 
////////////////////// Before //////////////////////
////////////////////// After ////////////////////// 
 0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object 
////////////////////// After //////////////////////


Top Movies Chart를 만들기위해 TMDB Ratings를 사용합니다. IMDB의 weighted rating 공식을 사용하여 차트를 구성합니다. 수학적으로는 다음과 같이 표현됩니다.

// TO-DO : 왜 수학적으로 이렇게 계산하는지?

Weighted Rating (WR) =  (vv+m.R)+(mv+m.C)

v : 영화에 대한 평가 수
m : 차트에 표시되어야 하는 최소 평가 수
R : 영화의 평점
C : 전체 영화에 대한 평균 점수
다음 단계는 차트에 표시되어야 하는 최소 투표 수 인 m에 적절한 값을 결정하는 것입니다. 95 번째 백분위 수를 컷오프로 사용합니다. 다시 말해서, 차트에 영화가 실리려면 영화가 목록에 있는 영화의 최소 95%보다 더 많은 표를 가져야합니다.

In [12]:
print('vote ::: \n', md[['vote_count', 'vote_average']].head())
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

vote ::: 
    vote_count  vote_average
0      5415.0           7.7
1      2413.0           6.9
2        92.0           6.5
3        34.0           6.1
4       173.0           5.7


5.244896612406511

In [13]:
# 총 45460개의 영화 중 상위 5%는 2273번째
print(vote_counts.sort_values(ascending=False)[2273:2274])

# quantile는 데이터를 크기대로 정렬하였을 때 분위수를 구하는 함수. quantile(0.95)는 상위 5%에 해당하는 값을 찾는 것
m = vote_counts.quantile(0.95)
m

11561    434
Name: vote_count, dtype: int64


434.0

In [14]:
print('release_date ::: \n', md['release_date'].head())

# pd.to_datetime
# errors : {‘ignore’, ‘raise’, ‘coerce’}, default ‘raise’
# If ‘raise’, then invalid parsing will raise an exception
# If ‘coerce’, then invalid parsing will be set as NaT
# If ‘ignore’, then invalid parsing will return the input

# 'release_date'를 split해서 year만 추출
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

print('year ::: \n', md['year'].head())

release_date ::: 
 0    1995-10-30
1    1995-12-15
2    1995-12-22
3    1995-12-22
4    1995-02-10
Name: release_date, dtype: object
year ::: 
 0    1995
1    1995
2    1995
3    1995
4    1995
Name: year, dtype: object


In [15]:
# 평가 수가 상위 5%인(434보다 큰) 데이터 추출
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

차트에 오를 수 있는 자격을 갖추려면 영화가 TMDB에서 최소 434 개 이상의 평가를 받아야합니다. 또한 TMDB에서 영화의 평균 평점은 10점 만점에 5.244입니다. 2274편의 영화가 우리 차트에 올랐습니다.

In [16]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [17]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [18]:
# Weighted Rating 상위 250개의 영화
qualified = qualified.sort_values('wr', ascending=False).head(250)

상위 250개 영화의 상단 15개 정보

In [19]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,"[Adventure, Fantasy, Action]",7.851924


# 콘텐츠 기반

In [21]:
links_small = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/Movie2/links_small.csv")
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
links_small.head()

0      862
1     8844
2    15602
3    31357
4    11862
Name: tmdbId, dtype: int64

In [22]:
# Drop a row by index : 19730, 29503, 33587 행은 이상한 데이터들(md.iloc[19730], md.iloc[29503], md.iloc[33587])
md = md.drop([19730, 29503, 35587])

In [23]:
#Check EDA Notebook for how and why I got these indices.
md['id'] = md['id'].astype('int')

기존 영화 데이터(45000편)의 1/5에 해당하는 9099편의 영화 데이터 셋 사용

In [24]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

## 영화 내용 기반 추천

## 영화 내용과 태그라인 사용

In [25]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

smd['description'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: description, dtype: object

In [26]:
# n-그램:단어장 생성에 사용할 토큰의 크기를 결정한다. 모노그램(1-그램)은 토큰 하나만 단어로 사용하며 바이그램(2-그램)은 두 개의 연결된 토큰을 하나의 단어로 사용한다.
# Stop Words:문서에서 단어장을 생성할 때 무시할 수 있는 단어를 말한다. 보통 영어의 관사나 접속사, 한국어의 조사 등이 여기에 해당한다. stop_words 인수로 조절할 수 있다.
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [27]:
print(tfidf_matrix[10])

  (0, 237880)	0.14118434571637967
  (0, 264587)	0.15239448296082492
  (0, 149397)	0.1346268358004416
  (0, 184592)	0.13757548758984955
  (0, 192607)	0.15239448296082492
  (0, 12515)	0.15239448296082492
  (0, 59410)	0.15239448296082492
  (0, 204261)	0.15239448296082492
  (0, 260369)	0.15239448296082492
  (0, 221301)	0.15239448296082492
  (0, 51585)	0.15239448296082492
  (0, 15732)	0.15239448296082492
  (0, 213978)	0.15239448296082492
  (0, 142130)	0.15239448296082492
  (0, 256956)	0.14583697304488685
  (0, 255009)	0.15239448296082492
  (0, 73255)	0.15239448296082492
  (0, 232769)	0.15239448296082492
  (0, 51809)	0.15239448296082492
  (0, 255952)	0.15239448296082492
  (0, 154810)	0.14583697304488685
  (0, 184597)	0.14583697304488685
  (0, 264425)	0.13213378623513553
  (0, 213982)	0.15239448296082492
  (0, 10509)	0.15239448296082492
  :	:
  (0, 259727)	0.15239448296082492
  (0, 237762)	0.08073572734141646
  (0, 192601)	0.13213378623513553
  (0, 12513)	0.13757548758984955
  (0, 59409)	0.14

In [28]:
tfidf_matrix.shape

(9099, 268124)

## 코사인 유사도

In [29]:
# linear_kernel는 두 벡터의 dot product 이다.
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [30]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

이제 데이터 세트의 모든 영화에 대해 pair 단위 코사인 유사도 매트릭스를 갖습니다. 다음 단계는 코사인 유사도 점수를 기반으로 가장 유사한 30개의 영화를 반환하는 함수를 작성하는 것입니다.

In [31]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

print(titles.head(), indices.head())

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [32]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [33]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [34]:
get_recommendations('Inception').head(10)

5239                              Cypher
141                                Crumb
6398                         Renaissance
653                            Lone Star
1703                               House
4739                    The Pink Panther
319                                 Cobb
2828    What Ever Happened to Baby Jane?
8867                     Pitch Perfect 2
979          Once Upon a Time in America
Name: title, dtype: object

In [35]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

## 메타 데이터 기반 추천

In [38]:
credits = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/Movie2/credits.csv")
keywords = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/Movie2/keywords.csv")

In [39]:
credits['crew'][0]

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

In [40]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [41]:
md.shape

(45463, 25)

In [42]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [43]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

이제 출연진, 제작진, 장르 및 크레딧이 모두 하나의 데이터 프레임에 있습니다. 아래의 직관을 사용하여 이것을 조금 더 풀어볼 것입니다.


Crew: 제작진 중 감독 외 다른 사람들은 영화의 분위기에 그다지 기여하지 않기 때문에, 제작진 데이터 중 감독만 feature로 선택할 것입니다.

Cast: 출연진 선택은 조금 더 까다롭습니다. 덜 알려진 배우와 비중이 작은 역할은 실제로 영화에 대한 사람들의 의견에 영향을 미치지 않습니다. 따라서 주연과 해당 배우만 선택해야합니다. 임의로 우리는 크레딧 목록에 나오는 상위 3명의 배우를 선택합니다.

In [44]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [45]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [46]:
smd['director'] = smd['crew'].apply(get_director)

In [47]:
# 출연진 중 상위에 노출되는 3명만 추출
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >= 3 else x)

In [48]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

모든 기능에서 스트립 스페이스 및 소문자로 변환. 이런 식으로 우리 엔진은 Johnny Depp와 Johnny Galecki를 혼동하지 않습니다.

감독은 3배로 언급하여은 전체 캐스트에 비해 가중치를 더 줍니다.

In [49]:
# 출연진의 이름에서 공백 삭제
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [50]:
# 감독의 이름에서 공백 삭제 및 3번 언급?
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x, x, x])

In [51]:
s = smd.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [52]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [53]:
# 2번 이상 등장한 키워드만 추출
s = s[s > 1]

In [54]:
# 어근 추출을 통해 동일 의미&다른 형태의 단어(dogs&dog, imaging&image 등)를 동일한 단어로 인식
stemmer = SnowballStemmer('english')
print("dogs의 어근 : ", stemmer.stem('dogs'))
print("dog의 어근 : ", stemmer.stem('dog'))

dogs의 어근 :  dog
dog의 어근 :  dog


In [55]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [56]:
# 키워드의 어근을 찾아서 공백 제거 후 세팅
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [57]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [58]:
count = CountVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [59]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [60]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [61]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
7648                     Inception
4145                      Insomnia
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

가중치 계산

In [62]:
def improved_recommendations(title):
    print(title)
    idx = indices[title]
    print(idx)
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    print(movie_indices)

    movies = smd.iloc[movie_indices][['title','vote_count','vote_average','year']]
#     print(movies)

    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull())]
#     print(qualified)
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    print(qualified)
    return qualified

In [63]:
improved_recommendations('The Dark Knight')

The Dark Knight
6981
[8031, 6218, 6623, 2085, 7648, 4145, 3381, 8613, 7659, 1134, 8927, 5943, 1260, 9024, 4021, 5809, 7362, 7561, 7582, 8001, 2754, 132, 2131, 2448, 5098]
                                   title  vote_count  vote_average  year  \
7648                           Inception       14075           8.1  2010   
8613                        Interstellar       11187           8.1  2014   
3381                             Memento        4168           8.1  2000   
6623                        The Prestige        4510           8.0  2006   
8031               The Dark Knight Rises        9263           7.6  2012   
6218                       Batman Begins        7511           7.5  2005   
1134                      Batman Returns        1706           6.6  1992   
9024  Batman v Superman: Dawn of Justice        7189           5.7  2016   
132                       Batman Forever        1529           5.2  1995   
1260                      Batman & Robin        1447           4.2  1

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8.1,2010,8.014597
8613,Interstellar,11187,8.1,2014,7.993373
3381,Memento,4168,8.1,2000,7.830744
6623,The Prestige,4510,8.0,2006,7.758148
8031,The Dark Knight Rises,9263,7.6,2012,7.494595
6218,Batman Begins,7511,7.5,2005,7.376814
1134,Batman Returns,1706,6.6,1992,6.325180
9024,Batman v Superman: Dawn of Justice,7189,5.7,2016,5.674090
132,Batman Forever,1529,5.2,1995,5.209926
1260,Batman & Robin,1447,4.2,1997,4.441087


In [76]:
improved_recommendations('Avatar')

Avatar
7488
[974, 522, 1011, 922, 4347, 344, 1376, 8401, 3216, 8724, 1500, 7265, 3060, 4966, 6084, 8419, 1668, 4017, 2132, 8658, 2761, 7088, 831, 1621, 2014]
                           title  vote_count  vote_average  year        wr
522   Terminator 2: Judgment Day        4274           7.7  1991  7.473680
974                       Aliens        3282           7.7  1986  7.413263
1376                     Titanic        7770           7.5  1997  7.380703
8658  X-Men: Days of Future Past        6155           7.5  2014  7.351462
8401     Star Trek Into Darkness        4479           7.4  2013  7.209624
1011              The Terminator        4208           7.4  1984  7.198510
922                    The Abyss         822           7.1  1989  6.458985
8419                Man of Steel        6462           6.5  2013  6.421010
344                    True Lies        1138           6.8  1994  6.370665
8724           Jupiter Ascending        2816           5.2  2015  5.205995


,title,vote_count,vote_average,year,wr
522,Terminator 2: Judgment Day,4274,7.7,1991,7.473680
974,Aliens,3282,7.7,1986,7.413263
1376,Titanic,7770,7.5,1997,7.380703
8658,X-Men: Days of Future Past,6155,7.5,2014,7.351462
8401,Star Trek Into Darkness,4479,7.4,2013,7.209624
1011,The Terminator,4208,7.4,1984,7.198510
922,The Abyss,822,7.1,1989,6.458985
8419,Man of Steel,6462,6.5,2013,6.421010
344,True Lies,1138,6.8,1994,6.370665
8724,Jupiter Ascending,2816,5.2,2015,5.205995


# 협업 필터링

In [64]:
# surprise 라이브러리의 Reader
reader = Reader()

In [65]:
ratings = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/Movie2/ratings_small.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [66]:
data = Dataset.load_from_df(ratings[['userId', 'movieId','rating']], reader)
# data.split(n_folds=5)

trainset = data.build_full_trainset()
testset = trainset.build_testset()

In [67]:
svd = SVD()
# evaluate(svd, data, measures=['RMSE', 'MAE'])

####### 기존 커널대로 진행하면 오류나서 수정 #######
svd.fit(trainset)
predictions = svd.test(testset)
accuracy.rmse(predictions)

RMSE: 0.6456


0.6455764230054826

In [68]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [78]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.6938073813680554, details={'was_impossible': False})

# 하이브리드


In [70]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [71]:
id_map = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset/Movie2/links_small.csv")[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [72]:
indices_map = id_map.set_index('id')

In [73]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']

    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    movies = smd.iloc[movie_indices][['title','vote_count','vote_average','year','id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [74]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.249501
974,Aliens,3282.0,7.7,1986,679,3.189122
1011,The Terminator,4208.0,7.4,1984,218,2.993042
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,2.884037
1621,Darby O'Gill and the Little People,35.0,6.7,1959,18887,2.883717
2014,Fantastic Planet,140.0,7.6,1973,16306,2.829691
922,The Abyss,822.0,7.1,1989,2756,2.771420
4347,Piranha Part Two: The Spawning,41.0,3.9,1981,31646,2.763988
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,2.735844
3060,Sinbad and the Eye of the Tiger,39.0,6.3,1977,11940,2.698712


In [75]:
hybrid(500, 'Avatar')

,title,vote_count,vote_average,year,id,est
1376,Titanic,7770.0,7.5,1997,597,3.676100
4347,Piranha Part Two: The Spawning,41.0,3.9,1981,31646,3.488557
1621,Darby O'Gill and the Little People,35.0,6.7,1959,18887,3.410773
8419,Man of Steel,6462.0,6.5,2013,49521,3.289578
1668,Return from Witch Mountain,38.0,5.6,1978,14822,3.256120
974,Aliens,3282.0,7.7,1986,679,3.248284
7088,Star Wars: The Clone Wars,434.0,5.8,2008,12180,3.230330
4966,Hercules in New York,63.0,3.7,1969,5227,3.208077
2014,Fantastic Planet,140.0,7.6,1973,16306,3.142157
1011,The Terminator,4208.0,7.4,1984,218,3.128396
